In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
data_path = "cifar-10-batches-py/data_batch_1"
batch1 = unpickle(data_path)

test_path = "cifar-10-batches-py/test_batch"
test = unpickle(test_path)

In [53]:
train_data = batch1[b'data']
train_labels = batch1[b'labels']

X_train = train_data
y_train = np.array(train_labels)

test_data = test[b'data']
test_labels = test[b'labels']

X_test_full = test_data
y_test_full = np.array(test_labels)

In [55]:
numbers = np.array(list(range(X_test_full.shape[0])))
indices = np.random.choice(numbers, X_test_full.shape[0]//2, replace=False)

other_indices = [x not in indices for x in numbers]

X_test = X_test_full[indices]
y_test = y_test_full.reshape(-1,)[indices] 

X_val = X_test_full[other_indices]
y_val = y_test_full.reshape(-1,)[other_indices]

In [56]:
height = 32
width = 32
channels = 3
n_inputs = height * width*channels

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

In [57]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 16 * 16])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [58]:
def fetch_batch(epoch, batch_size):
    np.random.seed(epoch * batch_size) 
    indices = np.random.randint(X_train.shape[0], size=batch_size) 
    X_batch = X_train[indices]
    y_batch = y_train[indices] 
    return X_batch, y_batch

In [60]:
n_epochs = 10
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            
            
            #X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch, y_batch = fetch_batch(epoch, batch_size)
            
            
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)
        
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Test accuracy:", acc_test)

0 Train accuracy: 1.0
0 Test accuracy: 0.138
1 Train accuracy: 1.0
1 Test accuracy: 0.165
2 Train accuracy: 1.0
2 Test accuracy: 0.1646
3 Train accuracy: 1.0
3 Test accuracy: 0.1698
4 Train accuracy: 1.0
4 Test accuracy: 0.1808
5 Train accuracy: 1.0
5 Test accuracy: 0.169
6 Train accuracy: 1.0
6 Test accuracy: 0.1768
7 Train accuracy: 1.0
7 Test accuracy: 0.1958
8 Train accuracy: 1.0
8 Test accuracy: 0.213
9 Train accuracy: 1.0
9 Test accuracy: 0.1612
